In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
# from torchsummary import summary
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
# from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdm

In [2]:
if torch.cuda.is_available():
    print("The code will run on GPU. This is important so things run faster.")
else:
    print("The code will run on CPU. You should probably not do this.")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The code will run on CPU. You should probably not do this.


In [3]:
class dataset:
    def __init__(self,data,target):
        self.data = data
        self.target = target
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        X = self.data[idx]
        y = self.target[idx]
        
        return X,y

In [4]:
data = np.load("armdata.npy")
data = data.reshape(100,100,3)

In [9]:
target = [1]*10+[2]*10+[3]*10+[4]*10+[5]*10+[6]*10+[7]*10+[8]*10+[9]*10+[10]*10
target = np.array(target)

In [ ]:
data = torch.from_numpy(data).float()
target = torch.from_numpy(target).float()

In [ ]:
trainset = 

In [ ]:
batch_size=5

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

In [11]:
class LinearNet(nn.Module):
    def __init__(self,years):
        super(LinearNet, self).__init__()
        
        self.linear = nn.Sequential(nn.Linear(in_features=300,out_features=600),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(600,400),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(400,200),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(200,100),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(100,50),
                                    nn.ReLU(),
                                    nn.Dropout(),
                                    nn.Linear(50,1),
                                    nn.Softmax())
                                      

        
    def forward(self, x): #x = [batch_size,100,3]
        x = x.view(x.shape[0],-1)
        x = self.linear(x)
        x = x.view(-1)
        return x


In [ ]:
model = LinearNet(years)
model.to(device)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)

critirion = nn.CrossEntropyLoss()

In [ ]:
#Training loop
num_epochs = 10
train_loss1 = []
test_loss1 = []

for epoch in tqdm(range(num_epochs), unit='epoch'):
    #For each epoch
    train_error1 = 0
    model.train()
    for minibatch_no, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
        data, target = data.to(device), target.to(device)
        #Zero the gradients computed for each weight
        optimizer.zero_grad()
        #Forward pass your image through the network
        output = model(data)
        #Compute the loss
        loss = critirion(output,target)

        #Backward pass through the network
        loss.backward()
        #Update the weights
        optimizer.step()
        train_error += loss
        
        
    #Comput the test accuracy
    model.eval()
    test_error = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_error += critirion(output,target)
        
    train_err1 = train_error1/len(trainset)
    train_loss1.append(train_err1)
    test_err1 = test_error1/len(testset)
    test_loss1.append(test_err1)

    print("train error 1: {train1:.1f}\t test error 1: {test1:.1f}".format(test1=test_err1,train1=train_err1))